### Manual calculations and files for preface.

In [2]:
import numpy as np
import h5py
import pathlib
from scipy.integrate import quad
from shared.preface import H0, Omega_M, Omega_L, s, SIM_DATA, ZEDS

def s_of_z(z):
    """Convert redshift to time variable s with eqn. 4.1 in Mertsch et al.
    (2020), keeping only Omega_m0 and Omega_Lambda0 in the Hubble eqn. for H(z).

    Args:
        z (float): redshift

    Returns:
        float: time variable s (in [seconds] if 1/H0 factor is included)
    """    

    def s_integrand(z):        

        # We need value of H0 in units of 1/s.
        H0_val = H0/(1/s)
        a_dot = np.sqrt(Omega_M*(1.+z)**3 + Omega_L)/(1.+z)*H0_val
        s_int = 1./a_dot

        return s_int

    s_of_z, _ = quad(s_integrand, 0., z)

    return np.float64(s_of_z)


sim = 'L006N188'

zeds = np.zeros(25)
nums = []
for j, i in enumerate(range(12,37)):
    snap_i = f'{i:04d}'
    nums.append(snap_i)

    with h5py.File(str(next(
        pathlib.Path(
            f'{SIM_DATA}/{sim}').glob(f'**/snapshot_{snap_i}.hdf5'
        )
    ))) as snap:
        zeds[j] = snap['Cosmology'].attrs['Redshift'][0]

np.save(f'shared/ZEDS_SNAPSHOTS.npy', np.asarray(zeds))
np.save(f'shared/NUMS_SNAPSHOTS.npy', np.asarray(nums))
print(np.round(np.asarray(zeds),2))

# Integration steps.
s_steps = np.array([s_of_z(z) for z in ZEDS])
np.save(f'shared/S_STEPS.npy', np.asarray(s_steps))

[3.81 3.43 3.09 2.79 2.52 2.28 2.06 1.86 1.68 1.51 1.36 1.21 1.08 0.96
 0.85 0.74 0.64 0.55 0.46 0.37 0.29 0.21 0.14 0.07 0.  ]
